# Imports and environment

In [1]:
import os
import sys
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI
from markdown_pdf import MarkdownPdf, Section

# Load environment variables (e.g. OPENAI_API_KEY)
load_dotenv()

True

# User‐configurable variables (formerly CLI args)

In [3]:
output_file   = r"..\results\Snowflake_comm_skills_dialogs.pdf"
model         = "gpt-4.1-mini"         # --model
toc_level     = 3                      # --toc-level
optimize      = False                  # --optimize
use_web_search= False                   # --no-web-search → False
max_output_tokens = 2000
prompts_filename = r"..\prompts\Snowflake_comm_skills_dialogs.txt"


# Helper function definitions

In [10]:
def generate_markdown_from_prompt(user_prompt, client):
    try:
        prompt = [
            {"role": "system", "content": (
                #"You are a coach helping me to improve my verbal communication skills. "
                "Provide output in Markdown. "
                "Use **bold** text for headers and not don't use '#' or '#'s headers. "
                "Don't add line separators in response. "
                # "Add code snippets where needed."
            )},
            {"role": "user", "content": (
                user_prompt + 
                "\n\n"
                )
             },
        ]
        tools = [{"type": "web_search_preview"}] if use_web_search else []
        response = client.responses.create(
            model=model,
            tools=tools,
            input=prompt,
            max_output_tokens = max_output_tokens
        )
        text = f"Prompt: {user_prompt}\n\n" + response.output_text.strip()
        return f"\n---\n{text}"
    except Exception as e:
        print(f"AI request failed for node '{user_prompt}': {e}")
        return f"{user_prompt}\n\n---\n\n"

def process_prompts(prompts_text, client, pdf, css):
    # lines = prompts_text.strip().splitlines()
    lines = [line for line in prompts_text.splitlines() if line.strip()]
    
    len_lines = len(lines)
    
    section_md = ""
    for idx, line in enumerate(lines, start=1):
        stripped = line.lstrip()
        # skip empty lines
        if not stripped:
            continue
        # skip lines starting with '#'
        elif stripped.startswith("#"):
            section_md = f"{section_md}\n\n{stripped}"
        else:
            print(f"Processing Line {idx}/{len_lines} ({round(idx*100/len_lines)}%)...")
            md = generate_markdown_from_prompt(stripped, client)
            #md = "Test"
            section_md = f"{section_md}\n\n{md}\n\n"
            pdf.add_section(Section(section_md), user_css=css)
            section_md = ""

# Prompts

In [11]:
# Open the file in read mode and assign its contents to chatgpt_prompts
with open(prompts_filename, "r", encoding="utf-8") as f:
    chatgpt_prompts = f.read()

# Optional: print to verify
print(chatgpt_prompts[:400])

# Snowflake Communication Skills Dialogs

## Sales & Negotiation Skills

### Pitching & Closing

1. Generate a dialog between A and B, in which A coaches B through a sales‐call scenario with a new prospect, focusing on improving B’s vocal variety and body‐language cues.
2. Generate a dialog between A and B, in which A role‐plays a cold call with B to a sales lead, emphasizing energy in B’s voice a


# Instantiate client, load YAML, and generate Markdown sections

In [12]:
from pathlib import Path

api_key = os.getenv("OPENAI_API_KEY")
css = Path("custom.css").read_text(encoding="utf-8")
pdf = MarkdownPdf(toc_level=toc_level, optimize=optimize)

if not api_key:
    print("Environment variable OPENAI_API_KEY is not set.")
    sys.exit(1)

client = OpenAI(api_key=api_key)
            
process_prompts(chatgpt_prompts, client, pdf, css)


Processing Line 4/66 (6%)...
Processing Line 5/66 (8%)...
Processing Line 6/66 (9%)...
Processing Line 7/66 (11%)...
Processing Line 8/66 (12%)...
Processing Line 9/66 (14%)...
Processing Line 10/66 (15%)...
Processing Line 11/66 (17%)...
Processing Line 14/66 (21%)...
Processing Line 15/66 (23%)...
Processing Line 16/66 (24%)...
Processing Line 17/66 (26%)...
Processing Line 18/66 (27%)...
Processing Line 20/66 (30%)...
Processing Line 21/66 (32%)...
Processing Line 22/66 (33%)...
Processing Line 23/66 (35%)...
Processing Line 26/66 (39%)...
Processing Line 27/66 (41%)...
Processing Line 28/66 (42%)...
Processing Line 29/66 (44%)...
Processing Line 30/66 (45%)...
Processing Line 33/66 (50%)...
Processing Line 34/66 (52%)...
Processing Line 35/66 (53%)...
Processing Line 36/66 (55%)...
Processing Line 38/66 (58%)...
Processing Line 39/66 (59%)...
Processing Line 42/66 (64%)...
Processing Line 43/66 (65%)...
Processing Line 44/66 (67%)...
Processing Line 45/66 (68%)...
Processing Line 4

# Save as PDF

In [13]:
try:
    pdf.save(output_file)
    print(f"PDF successfully saved to {output_file}")
except Exception as e:
    print(f"Failed to save PDF: {e}")
    raise

PDF successfully saved to ..\results\Snowflake_comm_skills_dialogs.pdf
